In [1]:
from src.features.mapping import SemanticMap3DBuilder
import numpy as np
import pathlib
import os
from yacs.config import CfgNode
from src.model.perception.perception_pipeline_config import get_perception_cfg
from src.model.perception.evaluation import get_ground_truth
from src.model.perception.labeler import LabelGenerator
from matplotlib import pyplot as plt
import torch
from src.model.perception.model_wrapper import ModelWrapper
from PIL import Image
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [2]:
if pathlib.PurePath(os.getcwd()).name == "notebooks":
    print(pathlib.PurePath(os.getcwd()).name)
    os.chdir("..")

notebooks


In [5]:
scene_id = '00800-TEEsavR23oF'
epoch = 0
TRAJECTORY_DIR = pathlib.Path('data') / 'interim' / 'trajectories' / '23-06-11,16:09:05' / f'epoch_{epoch}' / scene_id
SEMANTIC_INFO_PATH = (pathlib.Path('data') / 'raw' / 'minival' / 'scene_datasets' / 'hm3d' / 'minival' 
                      / scene_id / f'{scene_id.split("-")[1]}.semantic.txt')

In [18]:
semantics_list = []
rgb_list = []
truth_dicts = []
for i in [142, 143, 144]:
    semantics = np.load(TRAJECTORY_DIR / 'Semantic' / f'{i}.npy')
    semantics_list.append(semantics)
    rgb_list.append(Image.open(TRAJECTORY_DIR / 'RGB' / f'{i}.png'))
    truth_dicts.append(get_ground_truth(semantics, SEMANTIC_INFO_PATH))

In [24]:
from src.model.perception.perception_pipeline_config import get_perception_cfg
from src.data.MaskRCNNEvaluationDataset import MaskRCNNEvaluationDataset, eval_collate_fn
from pathlib import Path
from torch.utils.data import DataLoader


perception_cfg = get_perception_cfg()
perception_cfg.DATA_PATHS.TRAJECTORIES_DIR = str(Path(
        perception_cfg.DATA_PATHS.INTERIM_DATA_DIR, "trajectories", "23-06-11,16:09:05"))

eval_dataset = MaskRCNNEvaluationDataset(perception_cfg.DATA_PATHS, perception_cfg.DATA_GENERATOR.SPLIT, 0)
eval_dataloader = DataLoader(eval_dataset,
                                batch_size=perception_cfg.TRAINING.BATCH_SIZE,
                                shuffle=perception_cfg.TRAINING.SHUFFLE,
                                num_workers=perception_cfg.TRAINING.NUM_WORKERS,
                                collate_fn=eval_collate_fn)



In [ ]:
model_config = CfgNode()
model_config.USE_INITIAL_TRANSFORMS = True
model_config.SCORE_THRESHOLD = 0.5
model_config.MASK_THRESHOLD = 0.5
model_config.TRAINABLE_BACKBONE_LAYERS = 0
model = ModelWrapper(model_config)
model.cuda()

In [37]:
metrics = []

for i, (images, semantics, semantic_info_paths) in enumerate(eval_dataloader):
    ground_truths = [get_ground_truth(semantics[i], semantic_info_paths[i]) for i in range(len(images))]
    metrics.append(model.get_metrics(images, ground_truths))
    if i == 20:
        break

In [39]:
sum(metrics)/len(metrics)

0.14728284006317457